In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

In [2]:
train_essays = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
train_prompts = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv')
test_essays = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')

In [3]:
computer_words = ['the']

In [4]:
train_essays.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


In [5]:
train_prompts.head()

,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...


In [6]:
test_essays.head()

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [7]:
fcolumns = computer_words.copy()
fcolumns.append('generated')
fcolumns.append('id')
frequency_dt = pd.DataFrame([],columns=fcolumns)
for index,row in train_essays.iterrows():
    words = nltk.word_tokenize(row['text'])
    fd= nltk.FreqDist(words)
    tmp = {}
    for i in computer_words:tmp[i] = fd[i]
    tmp['generated'] = row['generated']
    tmp['id'] = row['id']
    frequency_dt.loc[len(frequency_dt)] = tmp
frequency_dt.head()

,the,generated,id
0,32,0,0059830c
1,13,0,005db917
2,22,0,008f63e3
3,24,0,00940276
4,25,0,00c39458


In [8]:
number = 5
counter_w = {}
counter_wC = {}
counter_C = 0
for index,row in frequency_dt.iterrows():
    if row['generated']:counter_C+=1
    for i in computer_words:
        if row[i]>=number:
            if i in counter_w.keys():counter_w[i]+=1
            else:counter_w[i] = 1
            if row['generated']:
                if i in counter_wC.keys():counter_wC[i]+=1
                else:counter_wC[i] = 1

In [9]:
total_docs = len(frequency_dt)
prob_C = counter_C / total_docs
prob_w= {}
prob_wC = {}
for key,val in counter_w.items():
    prob_w[key] = val/total_docs
    prob_wC[key] = counter_wC[key] / counter_C
print(prob_C)
print(prob_w)
print(prob_wC)

0.0021770682148040637
{'the': 0.9804063860667634}
{'the': 1.0}


In [10]:
def model(pa,db,dba):
    pb = 1
    for key,val in db.items():
        pb= pb * val
    pba = 1
    for key,val in dba.items():
        pba = pba*val
    
    pab = (pba*pa)/pb
    return pab

In [11]:
accuracy = model(prob_C,prob_w,prob_wC)
accuracy

0.0022205773501110288

In [12]:
test_dt = pd.DataFrame([],columns=['id','generated'])
for index,row in test_essays.iterrows():
    test_dt.loc[len(test_dt)] = [row['id'],accuracy]
test_dt.head()

,id,generated
0,0000aaaa,0.002221
1,1111bbbb,0.002221
2,2222cccc,0.002221


In [13]:
test_dt.to_csv('submission.csv')